In [1]:
import cursor
import torch

print(torch.__version__)
print(torch.cuda.is_available())

2.0.0+cu118
True


In [2]:
import pickle

with open('../data_storage/CBOW_run_1M_min_5_PP@2024-04-26-10-27-43_con5_pl1000000_emb64_ep1-track2idx.pkl', 'rb') as f:
    data_uris = pickle.load(f)

In [3]:
import torch

# Load the model
model_data = torch.load('../data_storage/CBOW_run_1M_min_5_PP@2024-04-26-10-27-43_con5_pl1000000_emb64_ep1.pt', map_location=torch.device('cpu'))

# Access embeddings and layers
embeddings = model_data['embedding.weight']

In [4]:
# Creating a dictionary of song uri's and their corresponding embeddings
embeddings_dict = {}

for uri, idx in iter(data_uris.items()):
    embeddings_dict[uri] = embeddings[idx]


In [5]:
# Average embedding calculated to use as PAD embedding

average_embedding = torch.mean(embeddings, dim=0)
average_embedding.unsqueeze(0)

tensor([[-0.2239, -0.1767, -0.1768,  0.0902,  0.1923,  0.0818, -0.1311, -0.2306,
         -0.1818, -0.2775,  0.1100,  0.2362,  0.1705, -0.1840,  0.2217,  0.1891,
         -0.1653, -0.2227, -0.2156, -0.2622,  0.1852, -0.2316, -0.2309, -0.2677,
         -0.2711,  0.1375,  0.2533, -0.2247, -0.2740, -0.2182, -0.1831, -0.0821,
         -0.2757, -0.0676, -0.1172,  0.1671, -0.1226, -0.2171, -0.0354,  0.2321,
         -0.2044,  0.0036,  0.2123,  0.2110, -0.1658, -0.2394,  0.2912,  0.1897,
         -0.2654, -0.1867, -0.1602,  0.1672, -0.2275, -0.1471, -0.2599,  0.1432,
         -0.2142,  0.2324,  0.2473, -0.0061,  0.1438,  0.1202,  0.2522,  0.0536]])

In [6]:
# creation of dataframe where each row holds a playlist id, name, songs and artists

import sqlite3
import pandas as pd

# # Connect to the SQLite database
# conn = sqlite3.connect('../data_storage/spotify.db')

# # Define the SQL query to retrieve playlist information
# sql_query = '''
#     SELECT playlist_track.pid, playlist.playlist_name, artist.artist_name, track.track_uri
#     FROM playlist_track
#     JOIN track ON playlist_track.track_uri = track.track_uri
#     JOIN album ON track.album_uri = album.album_uri
#     JOIN artist ON album.artist_uri = artist.artist_uri
#     JOIN playlist ON playlist_track.pid = playlist.pid
# '''

# # Execute the SQL query and load results into a DataFrame
# df = pd.read_sql_query(sql_query, conn)

# # Group by playlist ID and aggregate artists and songs into lists
# grouped = df.groupby(['pid', 'playlist_name']).agg({
#     'artist_name': lambda x: list(x),
#     'track_uri': lambda x: list(x)
# }).reset_index()

# # Rename columns for clarity
# grouped.rename(columns={'artist_name': 'artists', 'track_uri': 'songs'}, inplace=True)

# # Close the database connection
# conn.close()

# grouped.to_csv('back_up.csv', index=False)

import ast
grouped = pd.read_csv('back_up.csv')
grouped['artists'] = grouped['artists'].apply(ast.literal_eval)
grouped['songs'] = grouped['songs'].apply(ast.literal_eval)


In [7]:
# # Save time by saving dataframe to a CSV file.
# grouped.to_csv('back_up.csv', index=False)
# # read_from_csv = False

In [8]:
# Load artist embeddings from pickle file
with open("../data_storage/artist_embeddings.pkl", "rb") as f:
    artist_embeddings = pickle.load(f)

# Load playlist name embeddings from pickle file
with open("../data_storage/playlist_name_embeddings.pkl", "rb") as f:
    playlist_embeddings = pickle.load(f)

In [9]:
conn = sqlite3.connect('../data_storage/spotify.db')

# Define the SQL query to retrieve playlist information
sql_query = '''
    SELECT * FROM artist;
'''

# Execute the SQL query and load results into a DataFrame
artist_df = pd.read_sql_query(sql_query, conn)



In [24]:
artist_dict = artist_df.set_index('artist_name')['artist_uri'].to_dict()
# artist_embeddings['The Apryl Fool']
# artist_embeddings['spotify:artist:7hJcb9fa4alzcOq3EaNPoG']

In [32]:
from sklearn.model_selection import train_test_split
import random
import ast

#first clean dataset and remove playlists with < 15 songs??????
filtered_grouped = grouped[grouped['songs'].apply(len) >= 15]

def createDatasets(filtered_grouped, k, read_from_csv, embeddings_dict, average_embedding, artist_dict, artist_embeddings, playlist_embeddings_dict):
    # Split data into train and test sets
    train_df, test_df = train_test_split(filtered_grouped, test_size=0.2, random_state=42)

    # Function to process each subset (train/test)
    def process_data(df, read_from_csv, k):
        tensors = []  # Holds the tensor of concatenated song, artist, and playlist embeddings for each playlist
        remaining_songs = []  # Holds the remaining songs in the playlist to be predicted

        for _, row in df.iterrows():
            p_songs = row['songs']
            p_artists = row['artists']
            playlist_name = row['playlist_name']
            if read_from_csv and isinstance(p_songs, str):
                p_songs = ast.literal_eval(p_songs)
                p_artists = ast.literal_eval(p_artists)

            if len(p_songs) >= k:
                selected_songs = random.sample(p_songs, k)
                selected_artists = [p_artists[p_songs.index(song)] for song in selected_songs]

                playlist_embedding = playlist_embeddings_dict.get(playlist_name, torch.zeros_like(average_embedding))

                # Retrieve and concatenate artist embeddings to track embeddings, and add playlist embeddings
                p_embeddings = []
                for song, artist in zip(selected_songs, selected_artists):
                    song_embedding = embeddings_dict.get(song, average_embedding)
                    artist_uri = artist_dict.get(artist, None)
                    if artist_uri:
                        artist_embedding = torch.tensor(artist_embeddings.get(artist_uri, torch.zeros_like(average_embedding)))
                    else:
                        artist_embedding = torch.zeros_like(average_embedding)
                    
                    if not isinstance(song_embedding, torch.Tensor) or not isinstance(artist_embedding, torch.Tensor):
                        raise ValueError("Embedding is not a tensor.")
                    
                    combined_embedding = torch.cat((song_embedding, artist_embedding, playlist_embedding), dim=0)
                    p_embeddings.append(combined_embedding)

                tensors.append(torch.stack(p_embeddings, dim=0))
                remaining_songs.append([song for song in p_songs if song not in selected_songs])

        # Convert embeddings to tensors
        tensor = torch.stack(tensors, dim=0) if tensors else torch.tensor([])

        return tensor, remaining_songs

    # Process training data
    x_train, y_train = process_data(train_df, read_from_csv, k)
    
    # Process testing data
    x_test, y_test = process_data(test_df, read_from_csv, k)
    
    return x_train, y_train, x_test, y_test

# def createDatasets(filtered_grouped, k, read_from_csv, embeddings_dict, average_embedding):
#     # Split data into train and test sets
#     train_df, test_df = train_test_split(filtered_grouped, test_size=0.2, random_state=42)
#     
#     # Function to process each subset (train/test)
#     def process_data(df, read_from_csv, k):
#         embeddings = []  # Holds the embeddings of k songs in the playlist
#         remaining_songs = []  # Holds the remaining songs in the playlist to be predicted
#         
#         for _, row in df.iterrows():
#             p_songs = row['songs']
#             if read_from_csv and isinstance(p_songs, str):
#                 p_songs = ast.literal_eval(p_songs)
#             if len(p_songs) >= k:
#                 selected_songs = random.sample(p_songs, k)
#                 p_embeddings = [embeddings_dict.get(song, average_embedding) for song in selected_songs]
#                 embeddings.append(p_embeddings)
#                 remaining_songs.append([song for song in p_songs if song not in selected_songs])
#         
#         # Convert embeddings to tensors
#         tensors = []
#         for embedding_list in embeddings:
#             if embedding_list:  # Check if list is not empty
#                 tensor_list = [torch.tensor(embedding) for embedding in embedding_list]
#                 tensors.append(torch.stack(tensor_list, dim=0))
#         
#         # Stack tensors to create a single tensor for each subset
#         tensor = torch.stack(tensors, dim=0) if tensors else torch.tensor([])
# 
#         return tensor, remaining_songs
#     
#     # Process training data
#     x_train, y_train = process_data(train_df, read_from_csv, k)
#     
#     # Process testing data
#     x_test, y_test = process_data(test_df, read_from_csv, k)
#     
#     return x_train, y_train, x_test, y_test

In [11]:
import numpy as np

def r_precision(y_true, y_pred):
    # Convert y_true to a set for quick lookup -- Probably not needed
    y_true_set = set(y_true)

    # Determine the number of relevant items -- Not 100% sure
    r = len(y_true_set)
    
    # Filter y_pred to only include items that are in y_true -- Not 100% sure if this is correct
    relevant_predictions = [song for song in y_pred if song in y_true_set]
    
    # Calculate the number of relevant items that should be considered
    r_actual = min(r, len(relevant_predictions))
    
    # Count how many of the top-r_actual predictions are in the true set of relevant songs
    relevant_count = len(relevant_predictions[:r_actual])
    
    if r == 0:
        return 0  # Avoid division by zero if there are no relevant items
    return relevant_count / r

def recommended_songs_clicks(y_true, y_pred):
    # Convert ground truth list to set for quick look-up -- Again, probably not needed
    true_set = set(y_true)
    
    # Find the first relevant track in the predictions
    for i, track in enumerate(y_pred, start=1):
        if track in true_set:
            # how many blocks of 10 tracks are needed
            return (i - 1) // 10 + 1
    
    # If no relevant track is found, set a default value
    # Since the max number of clicks possible plus one is mentioned as 51
    return 51


def dcg(relevances, rank):
    relevances = np.array(relevances)
    if relevances.size:
        return relevances[0] + np.sum(relevances[1:] / np.log2(np.arange(2, relevances.size + 1)))
    return 0

def idcg(relevances):
    sorted_relevances = sorted(relevances, reverse=True)
    return dcg(sorted_relevances, len(sorted_relevances))

def ndcg(y_true, y_pred):
    relevances = [1 if song in y_true else 0 for song in y_pred]
    actual_dcg = dcg(relevances, len(relevances))
    ideal_dcg = idcg(relevances)
    if ideal_dcg == 0:
        return 0
    return actual_dcg / ideal_dcg

def playlist_loss(y_true, y_pred):
    total_loss = 0
    num_samples = len(y_true)
    
    for i in range(num_samples):
        # Convert y_true and y_pred to sets for quick lookup
        y_true_set = set(y_true[i])
        y_pred_set = set(y_pred[i])

        # Calculate R-Precision
        r_precision_value = r_precision(y_true_set, y_pred_set)

        # Calculate NDCG
        ndcg_value = ndcg(y_true_set, y_pred_set)

        # Calculate clicks
        clicks_value = recommended_songs_clicks(y_true_set, y_pred_set)

        # Define weights
        weight_r = 1
        weight_ndcg = 1
        weight_clicks = 1/50  # Adjusted weight for clicks

        # Calculate individual losses
        r_loss = 1 - r_precision_value
        ndcg_loss = 1 - ndcg_value

        # Subtract 1/50 to keep loss within the range of 0 to 3, as 1 click (1/50) is the minimum
        clicks_loss = clicks_value * weight_clicks - 1/50

        # Combine individual losses
        loss = (r_loss * weight_r) + (ndcg_loss * weight_ndcg) + clicks_loss

        # print(f'Loss: {loss} R loss: {r_loss} NDCG: {ndcg_loss} Clicks: {clicks_value}')

        # Accumulate total loss
        total_loss += loss
    
    # Calculate average loss
    average_loss = total_loss / num_samples
    
    return average_loss


In [57]:
import torch
import torch.optim as optim
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence

## embedding length 64
## lstm taking k track embeddings

# Initialize an empty dictionary to store the embeddings of songs that appear in more than 5 playlists
filtered_embeddings_dict = {}

# # Calculate playlist counts for each song URI
playlist_counts = {}
for songs_list in filtered_grouped['songs']:
    for song_uri in songs_list:
        playlist_counts[song_uri] = playlist_counts.get(song_uri, 0) + 1


# print(f'Playlist counts size: {len(playlist_counts)}')
# Filter embeddings dictionary to include only songs that appear in more than 5 playlists
for song_uri, embedding in embeddings_dict.items():
    # print(song_uri)
    if playlist_counts.get(song_uri, 0) > 400:
        filtered_embeddings_dict[song_uri] = embedding

print(len(filtered_embeddings_dict))

# filtered_embeddings_dict = {k: v.to(device) for k, v in embeddings_dict.items()}


22441


In [58]:
class ClosestEmbeddingsLayer(nn.Module):
    def __init__(self, num_closest=500):
        super(ClosestEmbeddingsLayer, self).__init__()
        self.num_closest = num_closest

    def forward(self, generated_embeddings):
        print(f'...Finding closest embeddings...')
        closest_embeddings_batch = []
        generated_embeddings = generated_embeddings.to('cpu')
        for generated_embedding in generated_embeddings:
            # Compute cosine similarity between the generated embedding and all embeddings in embeddings_dict
            similarities = {}
            # for key, value in embeddings_dict.items():
            for key, value in filtered_embeddings_dict.items():
                similarity = torch.nn.functional.cosine_similarity(generated_embedding.unsqueeze(0), value.unsqueeze(0))
                similarities[key] = similarity.item()
            
            # Sort the similarities and get the top num_closest embeddings
            closest_embeddings = sorted(similarities, key=similarities.get, reverse=True)[:self.num_closest]
            closest_embeddings_batch.append(closest_embeddings)
        
        return closest_embeddings_batch

class DynamicRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(DynamicRNN, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.closest_embeddings_layer = ClosestEmbeddingsLayer()

    def forward(self, x):
        print(f'...Model forward pass...')
        # Feed the input through the LSTM layer
        output, (hidden, cell) = self.lstm(x)
        # Applying the linear layer and closest embeddings layer on the last hidden state of the outputs
        output = self.fc(output[:, -1, :])  # Select the output at the last time step of each sequence directly
        closest_embeddings = self.closest_embeddings_layer(output)
        return closest_embeddings
    # def forward(self, x, lengths):
    #     print(f'...Model forward pass...')
    #     print(x)
    #     # Packing the padded sequence
    #     packed_input = nn.utils.rnn.pack_padded_sequence(x, lengths, batch_first=True, enforce_sorted=False)
    #     print(packed_input)
    #     packed_output, (hidden, cell) = self.lstm(packed_input)
    #     print(packed_output)
    #     # Unpacking the sequence
    #     output, _ = nn.utils.rnn.pad_packed_sequence(packed_output, batch_first=True)
    #     # Applying the linear layer and closest embeddings layer on the last hidden state of the outputs
    #     output = self.fc(output[range(len(output)), lengths - 1])  # Select the output at the last time step of each sequence
    #     closest_embeddings = self.closest_embeddings_layer(output)
    #     return closest_embeddings

In [59]:
def train_model_for_k(model, x_train, y_train, epochs, batch_size):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    # criterion = playlist_loss()  # Or any other appropriate loss function
    # closest_embeddings_layer = ClosestEmbeddingsLayer().to(device)

    # Convert datasets to tensors and move to the appropriate device
    x_train.to(device)
    # y_train = torch.tensor(y_train, dtype=torch.float).to(device)


    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for i in range(0, len(x_train), batch_size):
            # print(f'Batch: {i}')
            optimizer.zero_grad()
            batch_input = x_train[i:i+batch_size].to(device)
            batch_target = y_train[i:i+batch_size]
            print(f'Trained: {i} / {len(x_train)}')
            # lengths = torch.tensor([len(x) for x in batch_input], dtype=torch.int64)
            output = model(batch_input)
            # closest_embeddings = closest_embeddings_layer(output)
            #returns list of 500 closest embeddings
            loss = playlist_loss(batch_target, output)
            print(f'Loss: {loss}')
            loss_tensor = torch.tensor(loss, requires_grad=True, device=device)  # Convert loss to a tensor and move to CUDA device
            loss_tensor.backward()  # Compute gradients
            optimizer.step()  # Update weights
        
        print(f'Epoch {epoch+1}, Loss: {total_loss / (len(x_train) / batch_size)}')
        
        torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            }, 'model_checkpoint.pth')
    

def evaluate_model(model, x_test, y_test, batch_size):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for i in range(0, len(x_test), batch_size):
            batch_input = x_test[i:i+batch_size].to(device)
            batch_target = y_test[i:i+batch_size]
            lengths = torch.tensor([len(x) for x in batch_input], dtype=torch.int64)
            val_output = model(batch_input, lengths)
            # val_closest_embeddings = closest_embeddings_layer(val_output)
            val_loss = playlist_loss(batch_target, val_output)
            total_loss += val_loss

    print(f'Test Loss: {total_loss / (len(x_test) / batch_size)}')

# Example usage for each k
# print(f'Size of filtered dict: {len(embeddings_dict)}')
small_df = filtered_grouped.sample(frac=0.01)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
epochs = 5
batch_size = 64

# k_values = [5, 10, 20]
k_values = [10]
for k in k_values:
    x_train, y_train, x_test, y_test = createDatasets(small_df, k, True, filtered_embeddings_dict, average_embedding, artist_dict, artist_embeddings, playlist_embeddings)
    print(x_train.shape)
    model = DynamicRNN(input_size=192, hidden_size=128, output_size=64).to(device)
    train_model_for_k(model, x_train, y_train, epochs=epochs, batch_size=batch_size)
    with open('modelk10.pkl', 'wb') as f:
        pickle.dump(model, f)
    evaluate_model(model, x_test, y_test, batch_size)


# train_model_for_k(model, x_train, y_train, x_test, y_test, epochs=10, batch_size=32, k=k)

C:\Users\benhi\AppData\Local\Temp\ipykernel_2144\104190308.py:37: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  artist_embedding = torch.tensor(artist_embeddings.get(artist_uri, torch.zeros_like(average_embedding)))


torch.Size([7232, 10, 192])
Trained: 0 / 7232
...Model forward pass...
...Finding closest embeddings...


KeyboardInterrupt: 

In [ ]:
# from torch.nn.utils.rnn import pack_padded_sequence
# 
# k_values = [5, 10, 20]
# 
# # Initialize empty lists for data collection
# x_train_all, lengths_train_all, y_train_all = [], [], []
# x_test_all, lengths_test_all, y_test_all = [], [], []
# 
# # Collect and process datasets for each k
# for k in k_values:
#     x_train, lengths_train, y_train, x_test, lengths_test, y_test = createDatasets(small_df, k, True, filtered_embeddings_dict, average_embedding)
#     
#     # Append data without padding
#     x_train_all.extend(x_train)
#     x_test_all.extend(x_test)
#     
#     lengths_train_all.extend(lengths_train)
#     y_train_all.extend(y_train)
#     lengths_test_all.extend(lengths_test)
#     y_test_all.extend(y_test)
# 
# # Function to pad tensors to the same length and concatenate them
# def pad_and_concatenate(tensors):
#     max_length = max(tensor.size(0) for tensor in tensors)  # Find the global maximum length
#     padded_tensors = [torch.nn.functional.pad(tensor, (0, 0, 0, max_length - tensor.size(0))) for tensor in tensors]
#     return torch.cat(padded_tensors, dim=0)
# 
# # Pad and concatenate all sequence tensors
# x_train_all = pad_and_concatenate(x_train_all)
# x_test_all = pad_and_concatenate(x_test_all)
# 
# # Convert lists to tensors and concatenate
# lengths_train_all = torch.tensor(lengths_train_all)
# lengths_test_all = torch.tensor(lengths_test_all)
# 
# # Debug: Print sizes for verification
# print('Sizes:', x_train_all.size(0), lengths_train_all.size(0))
# 
# # Shuffle the training dataset before packing
# train_indices = torch.randperm(x_train_all.size(0))
# x_train_all = x_train_all[train_indices]
# lengths_train_all = lengths_train_all[train_indices]
# y_train_all = y_train_all[train_indices]
# 
# # Pack the padded sequences
# packed_x_train_all = pack_padded_sequence(x_train_all, lengths_train_all, batch_first=True, enforce_sorted=False)
# packed_x_test_all = pack_padded_sequence(x_test_all, lengths_test_all, batch_first=True, enforce_sorted=False)
# 
# 
# batch_size = 32
# epochs = 10
# 
# train_model_for_k(model,x_train_all, y_train_all, epochs, batch_size)
# 
# evaluate_model(model, x_test_all, y_test_all, batch_size)

    